In [1]:
import bhnerf
from astropy import units
import jax

import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
from tqdm.notebook import tqdm
from flax.training import checkpoints
from pathlib import Path
import ruamel.yaml as yaml

# Runing on 2 GPUs
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2,3'

# Import script function
import sys
sys.path.append('../scripts/')
from Fit_ALMA_LP_Apr11_SgrA_Flare import preprocess_data

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-e9z4sgp4 because the default path (/home/jovyan/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


Welcome to eht-imaging! v 1.2.2 



2023-06-20 16:54:50.062227: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /.singularity.d/libs


In [2]:
def sample_3D_recovery(checkpoint_dir, coords, chunk=-1):
    predictor = bhnerf.network.NeRF_Predictor.from_yml(checkpoint_dir)
    state = checkpoints.restore_checkpoint(checkpoint_dir, None)
    emission = bhnerf.network.sample_3d_grid(predictor.apply, state['params'], coords=coords, chunk=chunk)
    return emission

def image_plane_model(inc, spin, randomize_subpixel_rays=False):
    # Compute geodesics paths
    geos = bhnerf.kgeo.image_plane_geos(
        spin, inc, 
        num_alpha=num_alpha, 
        num_beta=num_beta, 
        alpha_range=[-fov_M/2, fov_M/2],
        beta_range=[-fov_M/2, fov_M/2],
        randomize_subpixel_rays=randomize_subpixel_rays
    )
    geos = geos.fillna(0.0)
     # Keplerian velocity and Doppler boosting
    rot_sign = {'cw': -1, 'ccw': 1}
    Omega = rot_sign[Omega_dir] * np.sqrt(geos.M) / (geos.r**(3/2) + geos.spin * np.sqrt(geos.M))
    umu = bhnerf.kgeo.azimuthal_velocity_vector(geos, Omega)
    g = bhnerf.kgeo.doppler_factor(geos, umu)

    # Magnitude normalized magnetic field in fluid-frame
    b = bhnerf.kgeo.magnetic_field_fluid_frame(geos, umu, **b_consts)
    domain = np.bitwise_and(np.bitwise_and(np.abs(geos.z) < z_width, geos.r > rmin), geos.r < rmax)
    b_mean = np.sqrt(np.sum(b[domain]**2, axis=-1)).mean()
    b /= b_mean

    # Polarized emission factors (including parallel transport)
    de_rot_model = np.deg2rad(de_rot_angle + 20.0)
    J = np.nan_to_num(bhnerf.kgeo.parallel_transport(geos, umu, g, b, Q_frac=Q_frac, V_frac=0), 0.0)
    J_rot = bhnerf.emission.rotate_evpa(J, de_rot_model)

    t_injection = -float(geos.r_o + fov_M/4)
    raytracing_args = bhnerf.network.raytracing_args(geos, Omega, t_injection, t_start_obs*units.hr, J_rot)
    return raytracing_args

def image_plane_model_perturb_rays(inc, spin):
    raytracing_args = []
    for i in tqdm(range(num_subrays), leave=False):
        raytracing_args.append(image_plane_model(inc, spin, randomize_subpixel_rays=True))
    return raytracing_args

def image_plane_fit(raytracing_args, checkpoint_dir, t, data, rmin=0.0, rmax=np.inf, batchsize=20):
    predictor = bhnerf.network.NeRF_Predictor.from_yml(checkpoint_dir)
    predictor.rmax = min(rmax, predictor.rmax)
    predictor.rmin = max(rmin, predictor.rmin)
    params = predictor.init_params(raytracing_args)
    state = predictor.init_state(params, checkpoint_dir=checkpoint_dir)
    train_step = bhnerf.optimization.TrainStep.image(t, data, sigma, dtype='lc')
    _, image_plane = bhnerf.optimization.total_movie_loss(batchsize, state, train_step, raytracing_args, return_frames=True)
    datafit = np.sum(((image_plane.sum(axis=(-1,-2)) - data) / sigma)**2) / len(t)
    return datafit, image_plane

# Data-fits
---
Data-fit as a function of inclination angle 

In [75]:
inc_loss_val = pd.read_csv('../checkpoints/alma/intrinsic_fits/vertical_b_variable_pixels1/inclination_loss_subrays_10_0.csv', index_col=0)
inc_loss_tor_val = pd.read_csv('../checkpoints/alma/intrinsic_fits/toroidal_b_variable_pixels1_new/inclination_loss_subrays_10_0.csv', index_col=0)
inc_loss_rad_val = pd.read_csv('../checkpoints/alma/intrinsic_fits/radial_b_variable_pixels1_new/inclination_loss_subrays_10_0.csv', index_col=0)
inc_loss_ccw_val = pd.read_csv('../checkpoints/alma/intrinsic_fits/vertical_b_variable_pixels1_ccw/inclination_loss_subrays_10_0.csv', index_col=0)

plt.rcParams.update({"text.usetex": True, "font.family": "Helvetica"})

%matplotlib widget
fig, axes = plt.subplots(1, 2, sharey=True, figsize=(10,4.8))
axes[0].errorbar(inc_loss_rad_val.index, np.nanmean(np.log10(inc_loss_rad_val), axis=1), np.nanstd(np.log10(inc_loss_rad_val), axis=1), 
             color='tab:green', marker='^', mfc='g', mec='g', 
             label=r'$\chi^2(\theta_{\rm o}|B_{r})$', markersize=5)
axes[0].axhline(np.nanmean(np.log10(inc_loss_rad_val), axis=1).min(), color='green', linestyle='--',linewidth=0.9)

axes[0].errorbar(inc_loss_tor_val.index, np.nanmean(np.log10(inc_loss_tor_val), axis=1), np.nanstd(np.log10(inc_loss_tor_val), axis=1), 
             color='tab:blue', marker='^', mfc='b', mec='b', 
             label=r'$\chi^2(\theta_{\rm o}|B_{\phi})$', markersize=5)
axes[0].axhline(np.nanmean(np.log10(inc_loss_tor_val), axis=1).min(), color='blue', linestyle='--',linewidth=0.9)

axes[0].errorbar(inc_loss_val.index, np.nanmean(np.log10(inc_loss_val), axis=1), np.nanstd(np.log10(inc_loss_val), axis=1), 
             color='tab:orange', marker='^', mfc='r', mec='r', 
             label=r'$\chi^2(\theta_{\rm o}|B_{z})$', markersize=5)
axes[0].axhline(np.nanmean(np.log10(inc_loss_val), axis=1).min(), color='red', linestyle='--',linewidth=0.9)


axes[1].errorbar(inc_loss_ccw_val.index, np.nanmean(np.log10(inc_loss_ccw_val), axis=1), np.nanstd(np.log10(inc_loss_ccw_val), axis=1), 
             color='tab:purple', marker='^', mfc='purple', mec='purple',
             label=r'$\chi^2(\theta_{\rm o}|{\tt CCW})$', markersize=5)
axes[1].axhline(np.nanmean(np.log10(inc_loss_ccw_val), axis=1).min(), color='purple', linestyle='--',linewidth=0.9)

axes[1].errorbar(inc_loss_val.index, np.nanmean(np.log10(inc_loss_val), axis=1), np.nanstd(np.log10(inc_loss_val), axis=1), 
             color='tab:orange', marker='^', mfc='r', mec='r', 
             label=r'$\chi^2(\theta_{\rm o}|{\tt CW})$', markersize=5)
axes[1].axhline(np.nanmean(np.log10(inc_loss_val), axis=1).min(), color='red', linestyle='--',linewidth=0.9)

for ax in axes:
    ax.axvline(12, color='black', linestyle='--',linewidth=0.8)
    ax.legend(loc='lower right', fontsize=14)
    ax.axhline(0, color='black', linestyle='--',linewidth=0.8)
    ax.set_xticks([12, 20, 40, 60, 80])
    ax.set_xlabel(r'Inclination [deg]', fontsize=16)
    ax.tick_params(axis='both', which='major', labelsize=14) 
    ax.set_ylim([0, 2.5])
    
axes[0].set_ylabel(r'$\log \chi^2$', fontsize=20)
axes[0].set_title(r'Validation: magnetic field configurations', fontsize=18)
axes[1].set_title(r'Validation: rotation direction', fontsize=18)
plt.tight_layout()
plt.savefig('alma/intrinsic_fits/vertical_b_variable_pixels1/ALMA_validation_fit_magnetic_configs_and_cw_vs_ccw.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [44]:
inc_loss_cw = pd.read_csv('../checkpoints/alma/intrinsic_fits/vertical_b_variable_pixels1/inclination_loss.csv', index_col=0)
inc_loss_ccw = pd.read_csv('../checkpoints/alma/intrinsic_fits/vertical_b_variable_pixels1_ccw/inclination_loss.csv', index_col=0)
inc_loss_vert = pd.read_csv('../checkpoints/alma/intrinsic_fits/vertical_b_variable_pixels1/inclination_loss.csv', index_col=0)
inc_loss_rad = pd.read_csv('../checkpoints/alma/intrinsic_fits/radial_b_variable_pixels1_new/inclination_loss.csv', index_col=0)
inc_loss_tor = pd.read_csv('../checkpoints/alma/intrinsic_fits/toroidal_b_variable_pixels1_new/inclination_loss.csv', index_col=0)
plt.rcParams.update({"text.usetex": True, "font.family": "Helvetica"})


%matplotlib widget
fig, axes = plt.subplots(1, 2, sharey=True, figsize=(9,4.8))
axes[0].errorbar(inc_loss_rad.index, np.nanmean(np.log10(inc_loss_rad), axis=1), np.nanstd(np.log10(inc_loss_rad), axis=1), 
             color='tab:green', marker='^', mfc='g', mec='g', 
             label=r'$\log \chi^2(\theta|B_{r})$', markersize=5)

axes[0].errorbar(inc_loss_tor.index, np.nanmean(np.log10(inc_loss_tor), axis=1), np.nanstd(np.log10(inc_loss_tor), axis=1), 
             color='tab:blue', marker='^', mfc='b', mec='b', 
             label=r'$\log \chi^2(\theta|B_{\phi})$', markersize=5)

axes[0].errorbar(inc_loss_vert.index, np.nanmean(np.log10(inc_loss_vert), axis=1), np.nanstd(np.log10(inc_loss_vert), axis=1), 
             color='tab:orange', marker='^', mfc='r', mec='r', 
             label=r'$\log \chi^2(\theta|B_{z})$', markersize=5)

axes[1].errorbar(inc_loss_ccw.index, np.nanmean(np.log10(inc_loss_ccw), axis=1), np.nanstd(np.log10(inc_loss_ccw), axis=1), 
             color='tab:purple', marker='^', mfc='purple', mec='purple',
             label=r'$\log \chi^2(\theta|{\tt CCW})$', markersize=5)

axes[1].errorbar(inc_loss_cw.index, np.nanmean(np.log10(inc_loss_cw), axis=1), np.nanstd(np.log10(inc_loss_cw), axis=1), 
             color='tab:orange', marker='^', mfc='r', mec='r', 
             label=r'$\log \chi^2(\theta|{\tt CW})$', markersize=5)


for ax in axes:
    ax.axvline(12, color='black', linestyle='--',linewidth=0.8)
    ax.legend(loc='lower right', fontsize=14)
    ax.axhline(0, color='black', linestyle='--',linewidth=0.8)
    ax.set_xticks([12, 20, 40, 60, 80])
    ax.set_xlabel(r'Inclination [deg]', fontsize=16)
    ax.tick_params(axis='both', which='major', labelsize=14) 
    
axes[0].set_title(r'Data-fit: magnetic field configurations', fontsize=18)
axes[1].set_title(r'Data-fit: rotation direction', fontsize=18)
plt.tight_layout()
plt.savefig('alma/intrinsic_fits/vertical_b_variable_pixels1/ALMA_data_fit_magnetic_configs_and_cw_vs_ccw.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
inclinations = np.arange(4, 82, 2, dtype=float)

inc_loss_vert = pd.read_csv('../checkpoints/alma/intrinsic_fits/vertical_b_variable_pixels1/inclination_loss.csv', index_col=0)
inc_loss_rad = pd.read_csv('../checkpoints/alma/intrinsic_fits/radial_b_variable_pixels1_new/inclination_loss.csv', index_col=0)
inc_loss_tor = pd.read_csv('../checkpoints/alma/intrinsic_fits/toroidal_b_variable_pixels1_new/inclination_loss.csv', index_col=0)
plt.rcParams.update({"text.usetex": True, "font.family": "Helvetica"})

%matplotlib widget
plt.figure(figsize=(5,4.8))
plt.axvline(12, color='black', linestyle='--',linewidth=0.8)
plt.errorbar(inclinations, np.nanmean(np.log10(inc_loss_rad), axis=1), np.nanstd(np.log10(inc_loss_rad), axis=1), 
             color='tab:green', marker='^', mfc='g', mec='g', 
             label=r'$\log \chi^2(\theta|B_{r})$', markersize=5)

plt.errorbar(inclinations, np.nanmean(np.log10(inc_loss_tor), axis=1), np.nanstd(np.log10(inc_loss_tor), axis=1), 
             color='tab:blue', marker='^', mfc='b', mec='b', 
             label=r'$\log \chi^2(\theta|B_{\phi})$', markersize=5)

plt.errorbar(inclinations, np.nanmean(np.log10(inc_loss_vert), axis=1), np.nanstd(np.log10(inc_loss_vert), axis=1), 
             color='tab:orange', marker='^', mfc='r', mec='r', 
             label=r'$\log \chi^2(\theta|B_{z})$', markersize=5)

plt.legend(loc='best', fontsize=14)
plt.axhline(0, color='black', linestyle='--',linewidth=0.8)

plt.xticks([12, 20, 40, 60, 80])
plt.xlabel(r'Inclination [deg]', fontsize=16)
plt.title(r'Data-fit: magnetic field configurations', fontsize=18)
plt.tight_layout()
plt.tick_params(axis='both', which='major', labelsize=14)

plt.savefig('alma/intrinsic_fits/vertical_b_variable_pixels1/magnetic_configurations.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
outpath = '/alma/intrinsic_fits/vertical_b_variable_pixels1/'
inclinations = np.arange(4, 82, 2, dtype=float)
spin_grid = np.linspace(1e-3, 0.99, 10)

inc_loss = pd.read_csv('../checkpoints/alma/intrinsic_fits/vertical_b_variable_pixels1/inclination_loss.csv', index_col=0)
plt.rcParams.update({"text.usetex": True, "font.family": "Helvetica"})

%matplotlib widget
plt.figure(figsize=(5,4.8))
ax1 = plt.gca()
ax2 = ax1.twiny()
ax2.errorbar(inclinations, np.nanmean(np.log10(inc_loss), axis=1), np.nanstd(np.log10(inc_loss), axis=1), 
             color='tab:orange', marker='^', mfc='r', mec='r', 
             label=r'\log \chi^2 $(\theta|B_z)$', markersize=5)

ax1.errorbar(spin_grid, np.nanmean(np.log10(spin_loss_subrays), axis=1), np.nanstd(np.log10(spin_loss_subrays), axis=1), 
             color='tab:blue', marker='^', mfc='b', mec='b', 
             label=r'validation $(a|\theta{=}12^\circ)$', markersize=5)

for ax in [ax1, ax2]:
    ax.tick_params(axis='both', which='major', labelsize=14)
    ax.legend(loc='best', fontsize=14)

plt.axhline(0, color='gray', linestyle='--',linewidth=0.8)
plt.axvline(12, color='gray', linestyle='--',linewidth=0.8)
ax2.set_xticks([12, 20, 40, 60, 80])
ax1.set_xlabel(r'Spin', fontsize=16)
ax2.set_xlabel(r'Inclination [deg]', fontsize=16)
# plt.title(r'Inclination / Spin data-fit: $\log \chi^2(\theta | {\bf w}^\star)$ / $\log \chi^2(a | \theta=12^\circ, {\bf w}^\star)$', fontsize=16)
plt.tight_layout()
# plt.savefig(outpath.joinpath('inclination_loss_w_subray_validation_w_spin_valonly.pdf'))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
inclinations = np.arange(4, 82, 2, dtype=float)
spin_grid = np.linspace(1e-3, 0.99, 10)

inc_loss = pd.read_csv(recovery_path.joinpath('inclination_loss.csv'), index_col=0)
inc_loss_subrays = pd.read_csv(recovery_path.joinpath('inclination_loss_subrays_10_0.csv'), index_col=0)
spin_loss = pd.read_csv('../checkpoints/alma/intrinsic_fits/vertical_b_spin_scan_rmin_ISCO/spin_loss.csv', index_col=0)
plt.rcParams.update({"text.usetex": True, "font.family": "Helvetica"})

%matplotlib widget
plt.figure(figsize=(5,4.8))
ax1 = plt.gca()
ax2 = ax1.twiny()
ax2.errorbar(inclinations, np.nanmean(np.log10(inc_loss), axis=1), np.nanstd(np.log10(inc_loss), axis=1), 
             color='tab:orange', marker='^', mfc='r', mec='r', 
             label=r'$\log \chi^2(\theta|a{=}0)$', markersize=5)
# ax1.errorbar(inclinations, np.nanmean(np.log10(inc_loss_subrays), axis=1), np.nanstd(np.log10(inc_loss_subrays), axis=1), 
#              color='tab:blue', marker='^', mfc='b', mec='b', 
#              label='validation', markersize=5)
ax1.errorbar(spin_grid, np.nanmean(np.log10(spin_loss), axis=1), np.nanstd(np.log10(spin_loss), axis=1), 
             color='tab:blue', marker='o', mfc='b', mec='b', 
             label=r'$\log \chi^2(a|\theta{=}12^\circ)$', markersize=4)

for ax in [ax1, ax2]:
    ax.tick_params(axis='both', which='major', labelsize=14)
    ax.legend(loc='best', fontsize=14)

plt.axhline(0, color='gray', linestyle='--',linewidth=0.8)
plt.axvline(12, color='gray', linestyle='--',linewidth=0.8)
ax2.set_xticks([12, 20, 40, 60, 80])
ax1.set_xlabel(r'Spin', fontsize=16)
ax2.set_xlabel(r'Inclination [deg]', fontsize=16)
# plt.title(r'Inclination / Spin data-fit: $\log \chi^2(\theta | {\bf w}^\star)$ / $\log \chi^2(a | \theta=12^\circ, {\bf w}^\star)$', fontsize=16)
plt.tight_layout()
plt.savefig(outpath.joinpath('inclination_loss_w_subray_validation_w_spin.pdf'))

# Visualize 3D recoveries
---

In [20]:
# 10 precent configuration
basename = 'inc_{:.1f}.seed_{}'
recovery_path = Path('../checkpoints/alma/intrinsic_fits/vertical_b_variable_pixels1_ccw//')
with open(recovery_path.joinpath('config.yml'), 'r') as stream:
    config = yaml.load(stream, Loader=yaml.Loader)

locals().update(config['model'])
rmax = fov_M / 2
if rmin == 'ISCO': rmin = float(bhnerf.constants.isco_pro(spin))


seeds = range(0, 4)
inc_grid = [6, 8, 10, 12, 14, 16]

zenith = np.deg2rad(35)

jit = True
resolution = 256
bh_radius = 1 + np.sqrt(1-spin**2)
cam_r = 55.
linewidth = 0.14
norm_const =  0.05

visualizer = bhnerf.visualization.VolumeVisualizer(resolution, resolution, resolution)

images = np.empty((len(seeds), len(inc_grid), resolution, resolution, 3))
emission = 0
visualizer.set_view(cam_r=cam_r, domain_r=rmax, azimuth=0.0, zenith=zenith)

for i, seed in enumerate(tqdm(seeds, desc='seed')):
    for j, inc in enumerate(tqdm(inc_grid, desc='inc', leave=False)):
        checkpoint_dir = recovery_path.joinpath(basename.format(inc, seed))
        emission = sample_3D_recovery(checkpoint_dir, visualizer.coords, chunk=32)
        images[i,j] = visualizer.render(emission / norm_const, facewidth=1.9*rmax, jit=jit, bh_radius=bh_radius, linewidth=linewidth).clip(a_max=1)

seed:   0%|          | 0/4 [00:00<?, ?it/s]

inc:   0%|          | 0/6 [00:00<?, ?it/s]

inc:   0%|          | 0/6 [00:00<?, ?it/s]

inc:   0%|          | 0/6 [00:00<?, ?it/s]

inc:   0%|          | 0/6 [00:00<?, ?it/s]

In [68]:
outpath = Path('alma/intrinsic_fits/vertical_b_variable_pixels1/')
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import Normalize

fig, axes = plt.subplots(3, 4, figsize=(9,6.5))
for i in range(2, 5):
    for j in range(4):
        axes[i-2,j].imshow(images[j,i])
        axes[i-2,j].set_xticks([])
        axes[i-2,j].set_yticks([])
        for spine in axes[i-2,j].spines.values():
            spine.set_visible(False)
        axes[i-2,0].set_ylabel(r'${}^\circ$'.format(inc_grid[i]), fontsize=16)
    axes[i-2,0].set_ylabel(r'${}^\circ$'.format(inc_grid[i]), fontsize=16)
# ax = fig.add_subplot(132)
# ax.set_visible(False)
# divider = make_axes_locatable(ax)
# cax = divider.append_axes('bottom', size='3%', pad=-1)
# cmap = plt.cm.ScalarMappable(norm=Normalize(0, norm_const, clip=True), cmap=plt.get_cmap('hot'))
# cbar = fig.colorbar(cmap, cax=cax, orientation='horizontal', shrink=.0)
# cbar.ax.tick_params(labelsize=12) 
plt.tight_layout()
# 
outname = '3D_Recovery_different_seeds_and_inc.pdf'
plt.savefig(outpath.joinpath(outname), bbox_inches='tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [55]:
# 10 precent configuration
seeds = [0, 1, 2, 3, 0, 1]
# inc_grid = np.arange(4, 24, 2, dtype=float)
inc_grid = [12, 12, 12, 12, 16, 16]

view_zeniths = [np.deg2rad(35), np.deg2rad(65)]

jit = False if len(view_zeniths) < 5 else True
resolution = 256
bh_radius = 1 + np.sqrt(1-spin**2)
cam_r = 55.
linewidth = 0.14
norm_const =  0.05

visualizer = bhnerf.visualization.VolumeVisualizer(resolution, resolution, resolution)

images = []
for zenith in tqdm(view_zeniths, desc='view angle'):
    emission = 0
    visualizer.set_view(cam_r=cam_r, domain_r=rmax, azimuth=0.0, zenith=zenith)
    for inc, seed in tqdm(zip(inc_grid, seeds), desc='inc/seed', leave=False):
        checkpoint_dir = recovery_path.joinpath(basename.format(inc, seed))
        emission += sample_3D_recovery(checkpoint_dir, visualizer.coords, chunk=32) / len(inc_grid)
        
    image = visualizer.render(emission / norm_const, facewidth=1.9*rmax, jit=jit, bh_radius=bh_radius, linewidth=linewidth).clip(a_max=1)
    images.append(image)

view angle:   0%|          | 0/2 [00:00<?, ?it/s]

inc/seed: 0it [00:00, ?it/s]

inc/seed: 0it [00:00, ?it/s]

In [69]:
seeds = [0]
# inc_grid = np.arange(4, 24, 2, dtype=float)
inc_grid = [12]

view_zeniths = [np.deg2rad(35), np.deg2rad(65)]

jit = False if len(view_zeniths) < 5 else True
resolution = 256
bh_radius = 1 + np.sqrt(1-spin**2)
cam_r = 55.
linewidth = 0.14
norm_const =  0.05

visualizer = bhnerf.visualization.VolumeVisualizer(resolution, resolution, resolution)

images = []
for zenith in tqdm(view_zeniths, desc='view angle'):
    emission = 0
    visualizer.set_view(cam_r=cam_r, domain_r=rmax, azimuth=0.0, zenith=zenith)
    for inc in tqdm(inc_grid, desc='inc', leave=False):
        for seed in tqdm(seeds, desc='seed', leave=False):
            checkpoint_dir = recovery_path.joinpath(basename.format(inc, seed))
            emission += sample_3D_recovery(checkpoint_dir, visualizer.coords, chunk=32) / (len(seeds)*len(inc_grid))
        
    image = visualizer.render(emission / norm_const, facewidth=1.9*rmax, jit=jit, bh_radius=bh_radius, linewidth=linewidth).clip(a_max=1)
    images.append(image)

view angle:   0%|          | 0/2 [00:00<?, ?it/s]

inc:   0%|          | 0/1 [00:00<?, ?it/s]

seed:   0%|          | 0/1 [00:00<?, ?it/s]

inc:   0%|          | 0/1 [00:00<?, ?it/s]

seed:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import Normalize

fig, axes = plt.subplots(1, len(images), figsize=(9,4))
for ax, image in zip(axes, images):
    ax.imshow(image)
    ax.set_axis_off()

ax = fig.add_subplot(132)
ax.set_visible(False)
divider = make_axes_locatable(ax)
cax = divider.append_axes('bottom', size='3%', pad=-1)
cmap = plt.cm.ScalarMappable(norm=Normalize(0, norm_const, clip=True), cmap=plt.get_cmap('hot'))
cbar = fig.colorbar(cmap, cax=cax, orientation='horizontal', shrink=.0)
cbar.ax.tick_params(labelsize=12) 
plt.tight_layout()

outname = '3D_Recovery_seeds_{}_incs_{}.pdf'.format('-'.join([str(seed) for seed in seeds]), '-'.join([str(int(inc)) for inc in inc_grid]))
plt.savefig(outpath.joinpath(outname), bbox_inches='tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Animate recoveries


In [ ]:
from matplotlib import animation

def animate_synced(movie1, movie2, loss, inc_grid, axes, cmap='RdBu_r', fps=10, output=None, writer='imagemagick'):

    def animate_both(i):
        return animate_frame(i), animate_plot(i)
        
    # Image animation function (called sequentially)
    def animate_frame(i):
        axes[0].set_title('Emission estimate: inc={:1.1f}'.format(inc_grid[i]))
        im1.set_array(movie1[i])
        
        axes[1].set_title('Emission estimate: inc={:1.1f}'.format(inc_grid[i]))
        im2.set_array(movie2[i])
        return im1, im2
    
    def animate_plot(i):
        line.set_xdata(inc_grid[i])
        return line,
    
    num_frames = len(movie1)
    
    fig = plt.gcf()
    axes[0].set_title('Emission estimate')
    axes[0].set_xticks([])
    axes[0].set_yticks([])
    
    im1 =  axes[0].imshow(np.zeros_like(movie1[0]))
    im2 =  axes[1].imshow(np.zeros_like(movie2[0]))
    axes[2].plot(inc_grid, loss)
    line = axes[2].axvline(0, color='green', linestyle='--', label='inc hypothesis')
    axes[2].set_title('Inclination Log(loss)')
    axes[2].legend(loc='upper left')
    
    plt.tight_layout()
    anim = animation.FuncAnimation(fig, animate_both, frames=num_frames, interval=1e3 / fps)

    if output is not None:
        anim.save(output, writer=writer, fps=fps)
    return anim

In [ ]:
checkpoint_dir_fmt = '../checkpoints/alma/intrinsic_fits/const_I/' + \
                      'IQU.sigmas_1e-01_1e-02_1e-02.spin_0.0.initkey{}.rmin6.0.z_width4.scale1.0.Qfrac{:.1f}.inc_{:1.1f}/'
inc_grid = np.deg2rad(np.linspace(2, 80, 40))
seeds = [2, 3, 4]
Q_frac = 0.85

In [ ]:
import warnings
warnings.simplefilter("ignore")

batchsize = 20
J_inds = [['I', 'Q', 'U'].index(s) for s in stokes]

loss = np.full((len(seeds), len(inc_grid)), fill_value=np.nan)

for i, seed in enumerate(tqdm(seeds, desc='seed')):
    for j, inclination in enumerate(tqdm(inc_grid, desc='inc', leave=False)):
        checkpoint_dir = checkpoint_dir_fmt.format(seed, Q_frac, np.rad2deg(inclination))
        if os.path.exists(checkpoint_dir):
            predictor = bhnerf.network.NeRF_Predictor.from_yml(checkpoint_dir)

            geos = bhnerf.kgeo.image_plane_geos(
                spin, inclination, 
                num_alpha=64, num_beta=64, 
                alpha_range=[-rmax, rmax],
                beta_range=[-rmax, rmax]
            )
            geos = geos.fillna(0.0)
            t_injection = -float(geos.r_o + fov_M/4)

            # Keplerian prograde velocity field
            Omega = rot_sign[Omega_dir] * np.sqrt(geos.M) / (geos.r**(3/2) + geos.spin * np.sqrt(geos.M))
            # Omega = rot_sign[Omega_dir] * np.sqrt(geos.M) / (11.0**(3/2) + geos.spin * np.sqrt(geos.M))

            umu = bhnerf.kgeo.azimuthal_velocity_vector(geos, Omega)
            g = bhnerf.kgeo.doppler_factor(geos, umu)
            b = bhnerf.kgeo.magnetic_field(geos, *b_consts)
            J = np.nan_to_num(bhnerf.kgeo.parallel_transport(geos, umu, g, b, Q_frac=Q_frac, V_frac=0), 0.0)[J_inds]
            J_rot = bhnerf.emission.rotate_evpa(J, de_rot_model)
            raytracing_args = bhnerf.network.raytracing_args(geos, Omega, t_injection, t_frames[0], J_rot)
            params = predictor.init_params(raytracing_args)
            state = predictor.init_state(params, checkpoint_dir=checkpoint_dir)
            loss[i, j] = bhnerf.optimization.total_movie_loss(batchsize, state, train_step, raytracing_args)

seed:   0%|          | 0/3 [00:00<?, ?it/s]

inc:   0%|          | 0/40 [00:00<?, ?it/s]

inc:   0%|          | 0/40 [00:00<?, ?it/s]

inc:   0%|          | 0/40 [00:00<?, ?it/s]

In [ ]:
%matplotlib widget
for i in range(len(seeds)):
    plt.scatter(np.rad2deg(inc_grid), loss[i],  marker='^')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "Helvetica"
})


In [ ]:
%matplotlib widget
plt.figure(figsize=(5,4))
plt.errorbar(np.rad2deg(inc_grid)[1:], np.nanmean(np.log(loss/nt), axis=0)[1:],  
             np.nanstd(np.log(loss/nt), axis=0)[1:], marker='^', mfc='r', mec='r')
plt.title(r'Inclination data-fit: $\log \chi^2(\theta | \hat{\bf w})$', fontsize=16)
plt.xticks(fontsize='14')
plt.yticks(fontsize='14')
plt.axhline(0, color='black', linestyle='--',linewidth=1)
plt.savefig('alma/inclination_loss.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
%matplotlib widget
plt.scatter(np.rad2deg(inc_grid), np.log(loss),  marker='^')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
import warnings
warnings.simplefilter("ignore")

batchsize = 20
stokes= ['Q', 'U']
target_qu = target[:,1:]
train_step = bhnerf.optimization.TrainStep.image(t_frames, target_qu, sigma[1:], dtype='lc')
J_inds = [['I', 'Q', 'U'].index(s) for s in stokes]

loss_qu = []
for inclination in tqdm(inc_grid, desc='inc'):
    
    checkpoint_dir = checkpoint_dir_fmt.format(np.rad2deg(inclination))
    if os.path.exists(checkpoint_dir):
        predictor = bhnerf.network.NeRF_Predictor.from_yml(checkpoint_dir)

        geos = bhnerf.kgeo.image_plane_geos(
            spin, inclination, 
            num_alpha=64, num_beta=64, 
            alpha_range=[-rmax, rmax],
            beta_range=[-rmax, rmax]
        )
        geos = geos.fillna(0.0)
        t_injection = -float(geos.r_o + fov_M/4)

        # Keplerian prograde velocity field
        Omega = rot_sign[Omega_dir] * np.sqrt(geos.M) / (geos.r**(3/2) + geos.spin * np.sqrt(geos.M))
        # Omega = rot_sign[Omega_dir] * np.sqrt(geos.M) / (11.0**(3/2) + geos.spin * np.sqrt(geos.M))

        umu = bhnerf.kgeo.azimuthal_velocity_vector(geos, Omega)
        g = bhnerf.kgeo.doppler_factor(geos, umu)
        b = bhnerf.kgeo.magnetic_field(geos, *b_consts)
        J = np.nan_to_num(bhnerf.kgeo.parallel_transport(geos, umu, g, b, Q_frac=Q_frac, V_frac=0), 0.0)[J_inds]
        J_rot = bhnerf.emission.rotate_evpa(J, de_rot_model)
        raytracing_args = bhnerf.network.raytracing_args(geos, Omega, t_injection, t_frames[0], J_rot)
        params = predictor.init_params(raytracing_args)
        state = predictor.init_state(params, checkpoint_dir=checkpoint_dir)

        loss_qu.append(bhnerf.optimization.total_movie_loss(batchsize, state, train_step, raytracing_args))
    else:
        loss_qu.append(np.nan)
    
loss_qu = np.array(loss_qu)

inc:   0%|          | 0/40 [00:00<?, ?it/s]

In [ ]:
checkpoint_dir_fmt = '../checkpoints/alma/intrinsic_fits/inc_scan_const_omega_r_11/' + \
                     'QU.spin_0.0.initkey2.rmin6.0.z_width4.inc_{:1.1f}/'

import warnings
warnings.simplefilter("ignore")

batchsize = 20
J_inds = [['I', 'Q', 'U'].index(s) for s in stokes]

loss_noshear = []
for inclination in tqdm(inc_grid, desc='inc'):
    checkpoint_dir = checkpoint_dir_fmt.format(np.rad2deg(inclination))
    predictor = bhnerf.network.NeRF_Predictor.from_yml(checkpoint_dir)
        
    geos = bhnerf.kgeo.image_plane_geos(
        spin, inclination, 
        num_alpha=64, num_beta=64, 
        alpha_range=[-rmax, rmax],
        beta_range=[-rmax, rmax]
    )
    geos = geos.fillna(0.0)
    t_injection = -float(geos.r_o)

    # Keplerian prograde velocity field
    # Omega = rot_sign[Omega_dir] * np.sqrt(geos.M) / (geos.r**(3/2) + geos.spin * np.sqrt(geos.M))
    Omega = rot_sign[Omega_dir] * np.sqrt(geos.M) / (11.0**(3/2) + geos.spin * np.sqrt(geos.M))
    
    umu = bhnerf.kgeo.azimuthal_velocity_vector(geos, Omega)
    g = bhnerf.kgeo.doppler_factor(geos, umu)
    b = bhnerf.kgeo.magnetic_field(geos, *b_consts)
    J = np.nan_to_num(bhnerf.kgeo.parallel_transport(geos, umu, g, b, Q_frac=Q_frac, V_frac=0), 0.0)[J_inds]
    J_rot = bhnerf.emission.rotate_evpa(J, de_rot_model)
    raytracing_args = bhnerf.network.raytracing_args(geos, Omega, t_injection, t_frames[0], J_rot)
    params = predictor.init_params(raytracing_args)
    state = predictor.init_state(params, checkpoint_dir=checkpoint_dir)
    
    loss_noshear.append(bhnerf.optimization.total_movie_loss(batchsize, state, train_step, raytracing_args))
    
loss_noshear = np.array(loss_noshear)

inc:   0%|          | 0/40 [00:00<?, ?it/s]

In [ ]:
%matplotlib widget

snr = 1
min_loss = np.rad2deg(inc_grid)[np.argmin(loss)]
plt.title('Inclination data-fit [log-scale]')
plt.scatter(np.rad2deg(inc_grid), np.log(loss * snr), label='With shear', marker='x')
plt.scatter(np.rad2deg(inc_grid), np.log(loss_noshear * snr), label='Without shear', marker='^')
plt.legend()
plt.savefig('alma/intrinsic fits/inc_loss_both.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Animate recovered volumes

In [ ]:
"""
Animate the recovered volumes from two view angles.
Note: this is time consuming to produce high res images.
"""
seed = 2
inc_grid = list(range(4, 22, 2))
from flax.training import checkpoints

jit = True
resolution = 256
bh_radius = 2.0
cam_r = 55.
linewidth = 0.1
bh_radius = 1 + np.sqrt(1-spin**2)
norm_const =  0.1
visualizer = bhnerf.visualization.VolumeVisualizer(resolution, resolution, resolution)
visualizer.set_view(cam_r=cam_r, domain_r=rmax, azimuth=0.0, zenith=np.deg2rad(35))

images1, images2 = [], []
for i, inclination in enumerate(tqdm(inc_grid)):
    checkpoint_dir = checkpoint_dir_fmt.format(seed, Q_frac, inclination)
    predictor = bhnerf.network.NeRF_Predictor.from_yml(checkpoint_dir)
    state = checkpoints.restore_checkpoint(checkpoint_dir, None)
    
    emission = bhnerf.network.sample_3d_grid(predictor.apply, state['params'], 
                                             coords=visualizer.coords, chunk=32)
    image = visualizer.render(emission / norm_const, facewidth=1.9*rmax, jit=jit, 
                              bh_radius=bh_radius, linewidth=linewidth)
    images1.append(image.clip(a_max=1))
    
    visualizer.set_view(cam_r=cam_r, domain_r=rmax, azimuth=0.0, zenith=np.deg2rad(65))
    emission = bhnerf.network.sample_3d_grid(predictor.apply, state['params'], 
                                             coords=visualizer.coords, chunk=32)
    image = visualizer.render(emission / norm_const, facewidth=1.9*rmax, jit=jit, 
                              bh_radius=bh_radius, linewidth=linewidth)
    images2.append(image.clip(a_max=1))

  0%|          | 0/9 [00:00<?, ?it/s]

IndexError: Replacement index 2 out of range for positional args tuple

In [ ]:
%matplotlib widget

fig, axes = plt.subplots(1, 3,figsize=(9,3.5))
output = 'alma/intrinsic fits/emission_estimate_1fps.mp4'
animate_synced(images, images_view2, np.log(loss), np.rad2deg(inc_grid), axes, output=output, fps=1, , writer='ffmpeg')

## Animate data fits across inclinations

In [ ]:
%matplotlib inline

import glob, warnings, subprocess
from pathlib import Path
import imageio

plt.style.use('default')

warnings.simplefilter("ignore")

directory = 'alma/datafit/'
frame_fmt = directory + 'frame{:03d}.png'
Path(directory).mkdir(parents=True, exist_ok=True)
gif_writer = imageio.get_writer(directory + 'datafit.gif', fps=1)
mp4_writer = imageio.get_writer(directory + 'datafit.mp4', fps=1)

stokes = ['Q', 'U']
J_inds = [['I', 'Q', 'U'].index(s) for s in stokes]
target = np.array(alma_lc_means[stokes])
sigma = 1.0
snr = 1.0
batchsize = 20
train_step = bhnerf.optimization.TrainStep.image(t_frames, target, sigma, dtype='lc')

for i, inclination in enumerate(tqdm(inc_grid, desc='inc')):
    checkpoint_dir = checkpoint_dir_fmt.format(np.rad2deg(inclination))
    
    geos = bhnerf.kgeo.image_plane_geos(
        spin, inclination, 
        num_alpha=64, num_beta=64, 
        alpha_range=[-rmax, rmax],
        beta_range=[-rmax, rmax]
    )
    geos = geos.fillna(0.0)
    t_injection = -float(geos.r_o)

    # Keplerian prograde velocity field
    Omega = rot_sign[Omega_dir] * np.sqrt(geos.M) / (geos.r**(3/2) + geos.spin * np.sqrt(geos.M))
    umu = bhnerf.kgeo.azimuthal_velocity_vector(geos, Omega)
    g = bhnerf.kgeo.doppler_factor(geos, umu)
    b = bhnerf.kgeo.magnetic_field(geos, *b_consts)
    J = np.nan_to_num(bhnerf.kgeo.parallel_transport(geos, umu, g, b, Q_frac=Q_frac, V_frac=0), 0.0)
    
    raytracing_args = bhnerf.network.raytracing_args(geos, Omega, t_injection, t_frames[0], J[J_inds])
    predictor = bhnerf.network.NeRF_Predictor.from_yml(checkpoint_dir)
    params = predictor.init_params(raytracing_args)
    state = predictor.init_state(params, checkpoint_dir=checkpoint_dir)
    
    _, movie = bhnerf.optimization.total_movie_loss(batchsize, state, train_step, raytracing_args, True)
    lc_est = movie.sum(axis=(-1,-2))
    
    fig, axes = plt.subplots(1, 4, figsize=(14, 3.5))
    axes[3].set_title(r'Inclination log(loss): ${:1.1f}^\circ$'.format(np.rad2deg(inclination)))
    axes[3].plot(np.rad2deg(inc_grid), np.log(loss * snr))
    axes[3].axvline(np.rad2deg(inclination), color='green', linestyle='--', label='inc hypothesis')
    bhnerf.visualization.plot_stokes_lc(target, stokes, t_frames, axes=axes[:3], label='True')
    bhnerf.visualization.plot_stokes_lc(lc_est, stokes, t_frames, axes=axes[:3], color='r', fmt='x', label='Estimate')
    for ax in axes:
        ax.legend()
        
    plt.savefig(frame_fmt.format(i), dpi=100)
    plt.close()
    
    im = imageio.imread(frame_fmt.format(i))
    gif_writer.append_data(im)
    mp4_writer.append_data(im)
    
gif_writer.close()
mp4_writer.close()

inc:   0%|          | 0/40 [00:00<?, ?it/s]